In [2]:
import pandas as pd

In [13]:
#api활용을 위해 제공되고 있는 xycode에서 부산 code만 추출한다

In [3]:
data = pd.read_excel('../Data/rawdata/API_xycode.xlsx')

In [5]:
data['CODE'] = data['행정구역코드'].apply(str)

In [6]:
busan_data = data[data['CODE'].str.startswith('26')]

In [7]:
busan_data = busan_data[['2단계','3단계','격자 X', '격자 Y', 'CODE']]

In [8]:
busan_data = busan_data.fillna(0)
busan_data = busan_data[busan_data['3단계']==0]

In [9]:
busan_data = busan_data.drop(columns='3단계')
busan_data = busan_data.drop(index=451)
busan_data

,2단계,격자 X,격자 Y,CODE
452,중구,97,74,2611000000
462,서구,97,74,2614000000
476,동구,98,75,2617000000
489,영도구,98,74,2620000000
501,부산진구,97,75,2623000000
522,동래구,98,76,2626000000
536,남구,98,75,2629000000
554,북구,96,76,2632000000
568,해운대구,99,75,2635000000
587,사하구,96,74,2638000000


In [10]:
busan_data = busan_data.rename(columns={'2단계':'ZONE','격자 X':'X','격자 Y' : 'Y'})

In [11]:
busan = busan_data.reset_index(drop=True)

In [12]:
busan

,ZONE,X,Y,CODE
0,중구,97,74,2611000000
1,서구,97,74,2614000000
2,동구,98,75,2617000000
3,영도구,98,74,2620000000
4,부산진구,97,75,2623000000
5,동래구,98,76,2626000000
6,남구,98,75,2629000000
7,북구,96,76,2632000000
8,해운대구,99,75,2635000000
9,사하구,96,74,2638000000


In [14]:
busan_data.to_csv('../Data/final_data/DB/xycode.csv', encoding='utf-8', index=False)

In [ ]:
# busan_dong을 위한 코드를 만든다.

In [17]:
busan_dong = pd.read_csv('../Data/processing_data/1st/busan_flood_geo.csv')

In [19]:
busan_dong.columns

Index(['SEQ', 'FLUD_CAUSE', 'F_DSTRC_NM', 'F_DSSTR_NM', 'F_CUE_DTL', 'ADM_CD',
       'WRITNG_YM', 'FLUD_TIME', 'FLUD_DPWT', 'DMG_AMOUNT', 'F_BEGIN_DE',
       'F_END_DE', 'F_BEGIN_TM', 'F_END_TM', 'FLUD_WAL', 'CITY', 'ZONE',
       'DONG', 'FLUD_DPWT.1', 'SLOPE_L', 'SLOPE_H', 'SLOPE_AVG', 'HIGH'],
      dtype='object')

In [20]:
busan_dong = busan_dong[['ADM_CD','ZONE','DONG','SLOPE_AVG','HIGH','FLUD_DPWT.1']]

In [25]:
busan_dong = busan_dong.sort_values('FLUD_DPWT.1', ascending=False,ignore_index=True)

In [32]:
busan_dong = busan_dong.drop_duplicates(['DONG'], keep='first')\
.sort_values(['ZONE','DONG'], ascending=['FALSE','TRUE'])\
.reset_index(drop=True)

In [36]:
busan_dong = busan_dong.drop(columns='FLUD_DPWT.1')

In [40]:
new_data={'ADM_CD':2617053000,'ZONE':'동구','DONG':'초량동','SLOPE_AVG':'1','HIGH':'4'}
busan_dong = busan_dong.append(new_data, ignore_index=True)

In [43]:
busan_dong.to_csv('../Data/final_data/DB/base_data.csv', encoding='utf-8', index=False)

In [45]:
fuck = pd.read_csv('../Data/')

,ADM_CD,ZONE,DONG,SLOPE_AVG,HIGH
0,2644012000,강서구,눌차동,22.5,9.11
1,2644000000,강서구,대저1동,1,1.77
2,2644000000,강서구,대저2동,1,1.18
3,2644010400,강서구,명지동,1,1.55
4,2644011900,강서구,성북동,11,1.31
5,2641000000,금정구,구서동,11,24.95
6,2641000000,금정구,금사동,4.5,6.88
7,2641000000,금정구,남산동,4.5,36.22
8,2641000000,금정구,노포동,11,63.77
9,2641000000,금정구,두구동,1,61.97


In [44]:
########

In [71]:
base_geo_data.to_csv('base_data.csv', encoding='utf-8')

In [73]:
busan_data = busan_data.reset_index()
busan_data = busan_data.drop(columns='index')
busan_data

,ZONE,X,Y,CODE
0,중구,97,74,2611000000
1,서구,97,74,2614000000
2,동구,98,75,2617000000
3,영도구,98,74,2620000000
4,부산진구,97,75,2623000000
5,동래구,98,76,2626000000
6,남구,98,75,2629000000
7,북구,96,76,2632000000
8,해운대구,99,75,2635000000
9,사하구,96,74,2638000000
